In [6]:
import tensorflow as tf
import numpy as np
import os
import shutil
from tensorflow.contrib.tensorboard.plugins import projector

In [7]:
def prepare_tensorboard(log_dir, npy_file, meta_file):
    a = np.load(npy_file)
    v = tf.Variable(a.reshape(len(a),100), name='user_embedding')
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    shutil.copy(meta_file, os.path.join(log_dir, 'metadata.tsv'))
    
    config = projector.ProjectorConfig()

    # You can add multiple embeddings. Here we add only one.
    embedding = config.embeddings.add()
    embedding.tensor_name = v.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = 'metadata.tsv'

    with tf.Session() as sess:
        #     embedding_var = tf.Variable(tf.pack([tf.squeeze(x) for x in outputs], axis=0), trainable=False, name='pool3')
        sess.run(tf.variables_initializer([v]))
        saver = tf.train.Saver([v])
        saver.save(sess,  os.path.join(log_dir, "model.ckpt"))

    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter(log_dir)

    # The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
    # read this file during startup.
    projector.visualize_embeddings(summary_writer, config)

In [8]:
prepare_tensorboard('test', '../DecisionTree/embed_old.npy', '../DecisionTree/cluster_old.tsv')

In [9]:
prepare_tensorboard('tmp', '../DecisionTree/Ikegami/embed.npy', '../DecisionTree/Ikegami/cluter.tsv')

In [38]:

LOG_DIR = 'tmp'
 
# Create randomly initialized embedding weights which will be trained.
N = 10000 # Number of items (vocab size).
D = 200 # Dimensionality of the embedding.
embedding_var = tf.Variable(tf.random_normal([N,D]), name='word_embedding')
 
# Format: tensorflow/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()
 
# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
# Link this tensor to its metadata file (e.g. labels).
# embedding.metadata_path = 'model.ckpt.meta'

with tf.Session() as sess:
    #     embedding_var = tf.Variable(tf.pack([tf.squeeze(x) for x in outputs], axis=0), trainable=False, name='pool3')
    sess.run(tf.variables_initializer([embedding_var]))
    saver = tf.train.Saver([embedding_var])
    saver.save(sess,  os.path.join(LOG_DIR, "model.ckpt"))

 
# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)
 
# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

In [ ]:

import sys
sys.path.append('../tf_models/tutorials/image/imagenet/')
import classify_image

In [ ]:
class flags:
    model_dir = 'tf_image'

In [ ]:
classify_image.FLAGS = flags

In [15]:
classify_image.maybe_download_and_extract()
classify_image.create_graph()